In [6]:
# test realtime pose CLASSIFICATION (and hence landmark detection...)
# on our curated dataset of pose videos

import sys
sys.path.insert(0, '/Users/alejandraduran/Documents/Pton_courses/COS429/COS429_final_project/training_pipeline')
import os
import cv2
import mediapipe as mp
import pickle
from extract_features import FeaturesMP
import numpy as np
import time
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

# Retrieve pre-trained model
mp_model_path = "/Users/alejandraduran/Documents/Pton_courses/COS429/COS429_final_project/pretrained_models/pose_landmarker_full.task"
# Initialize FeaturesMP object
features_mp = FeaturesMP(mp_model_path, image_size=(1080, 1920))
# load the label encoder
with open('/Users/alejandraduran/Documents/Pton_courses/COS429/COS429_final_project/training_pipeline/label_encoder.pkl', 'rb') as f:    
    label_encoder = pickle.load(f)

# NOT SURE IF 7 OR 8 BUT ONE OF THOSE
# load the trained classifier
with open('/Users/alejandraduran/Documents/Pton_courses/COS429/COS429_final_project/trained_classifiers/padded_nn_8.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
# load the sanskrit to english dictionary
with open('/Users/alejandraduran/Documents/Pton_courses/COS429/COS429_final_project/sanskrit_english_dict.pkl', 'rb') as f:
    sanskrit_english_dict = pickle.load(f)


# function to write demonstration videos - rest of testing function is sent in test_video.py

def test_video(video_path, features_mp, label_encoder, classifier, sanskrit_english_dict):

    cap = cv2.VideoCapture(video_path)

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    n = -1

    # introduce delay in position predictions
    buffer = [-1,-1,-1,-1]

    label_across_frames = []
    label_with_delays = []

    # Initialize detector
    features_mp.init_detector(video=True, min_pose_detection_confidence=0.7)

    # Create a loop to read the latest frame from the camera
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Unable to fetch the frame or finished the video.")
            break
        
        # new frame
        n+=1
        # get timestamp from frame rate
        timestamp = int(n * 1000 / frame_rate)

        # Run inference on the image 
        landmarks = features_mp.detect(frame, video=True, frame_timestamp_ms=timestamp)
        
        # Draw landmarks if detected
        if landmarks is not None:
            if len(landmarks.pose_landmarks) != 0:
                
                pose_landmarks_list = landmarks.pose_landmarks 

                # get only normalized coordinates - improves latency
                pose_landmarks = pose_landmarks_list[0]

                to_classify = np.zeros((features_mp.n_landmarks, 4))
                to_extend = []
                
                # store normalized landmarks to appends and classify
                for k, landmark in enumerate(pose_landmarks):
                    to_classify[k] = [landmark.x, landmark.y, landmark.z, landmark.visibility]
                    
                # normalize and rotate to_classify
                to_classify = features_mp.make_rot_invariant_partial(to_classify, init_norm=True)
                to_classify = to_classify.reshape(1, features_mp.n_landmarks * 4)
                    
                # Run inference
                predicted_class = classifier.predict(to_classify)
                # Get the string label
                predicted_name = label_encoder.inverse_transform([int(predicted_class-1)])
                english = sanskrit_english_dict[predicted_name[0]]
                label_across_frames.append(english)
                # Append to buffer
                buffer.pop(0)
                buffer.append(predicted_name[0])
                # # if all elements now in the buffer are the same, then we can display the pose
                if buffer[0] == buffer[1] == buffer[2] == buffer[3]:
                    text = english
                    label_with_delays.append(english)

    # Release resources
    cap.release()
    # out.release()
    cv2.destroyAllWindows()
    
    return label_across_frames, label_with_delays


In [8]:
label_accross_frames_list = []
label_with_delays_list = []
    
directory_path = "/Users/alejandraduran/Documents/Pton_courses/COS429/video_data/Adho Mukha Svanasana"
# iterate over the videos of the directory
for dirname, _, filenames in os.walk(directory_path):
    for filename in filenames:
        if not filename.endswith(('.mp4', '.avi')):  # Only process video files
            continue  # Skip non-video files
        video_path = os.path.join(dirname, filename)
        label_across_frames, label_with_delays = test_video(video_path, features_mp, label_encoder, classifier, sanskrit_english_dict)
        label_accross_frames_list.append(label_across_frames)
        label_with_delays_list.append(label_with_delays)
        print('\n\n DONE WITH:', filename)

I0000 00:00:1733953626.503393 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 6.mp4


I0000 00:00:1733953645.578249 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 4.mp4


I0000 00:00:1733953653.795350 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


...

Error: Unable to fetch the frame or finished the video.


 DONE WITH: 13.mp4


I0000 00:00:1733953883.376190 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 8.mp4


I0000 00:00:1733953895.597421 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 9.mp4


In [9]:
from collections import Counter

# Flatten the list of lists
flattened_list_accross_frames = [item for sublist in label_accross_frames_list for item in sublist]
flattened_list_with_delays = [item for sublist in label_with_delays_list for item in sublist]

# Count unique terms
counter_accross_frames = Counter(flattened_list_accross_frames)
counter_with_delays = Counter(flattened_list_with_delays)

# Print the counts
print("Counts for label_accross_frames_list:")
print(counter_accross_frames)

print("Counts for label_with_delays_list:")
print(counter_with_delays)

Counts for label_accross_frames_list:
Counter({'Butterfly': 3057, 'Plow': 2584, 'Plank': 362, 'Handstand': 232, 'Chair': 180, 'Forearm Stand': 154, 'Extended Side Angle': 129, 'Lotus': 123, 'Warrior I': 100, 'Reverse Warrior': 58, 'Shoulder Stand': 31, 'High Lunge': 25, 'Bridge': 24, 'Half-Boat': 24, 'Standing Forward Bend': 20, 'King Pigeon': 16, 'Splits': 16, 'Cow': 16, 'Corpse': 15, 'Boat': 14, 'Pyramid': 13, 'Camel': 9, 'Tree': 6, 'Extended Hand to Toe': 6, 'Triangle': 4, 'Sphinx': 3, 'Warrior III': 3, "Child's Pose": 2, 'Eagle': 2, 'Dolphin': 2, 'Cat': 1, 'Low Lunge': 1, 'Upward-Facing Dog': 1, 'Warrior II': 1})
Counts for label_with_delays_list:
Counter({'Butterfly': 2895, 'Plow': 2496, 'Plank': 269, 'Handstand': 178, 'Chair': 138, 'Forearm Stand': 135, 'Extended Side Angle': 64, 'Lotus': 59, 'Shoulder Stand': 18, 'Warrior I': 16, 'Reverse Warrior': 16, 'Half-Boat': 14, 'Cow': 9, 'Standing Forward Bend': 8, 'Splits': 6, 'Boat': 6, 'King Pigeon': 3, 'Bridge': 2, 'High Lunge': 1})


In [10]:
label_accross_frames_list = []
label_with_delays_list = []
    
directory_path = "/Users/alejandraduran/Documents/Pton_courses/COS429/video_data/Adho Mukha Vrksasana"
# iterate over the videos of the directory
for dirname, _, filenames in os.walk(directory_path):
    for filename in filenames:
        if not filename.endswith(('.mp4', '.avi')):  # Only process video files
            continue  # Skip non-video files
        video_path = os.path.join(dirname, filename)
        label_across_frames, label_with_delays = test_video(video_path, features_mp, label_encoder, classifier, sanskrit_english_dict)
        label_accross_frames_list.append(label_across_frames)
        label_with_delays_list.append(label_with_delays)
        print('\n\n DONE WITH:', filename)

I0000 00:00:1733953909.117984 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 4.mp4


I0000 00:00:1733953923.823711 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 5.mp4


I0000 00:00:1733953941.646180 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


...

Error: Unable to fetch the frame or finished the video.


 DONE WITH: 3.mp4


I0000 00:00:1733953987.494448 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 8.mp4


I0000 00:00:1733954003.492914 94137244 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1


Error: Unable to fetch the frame or finished the video.


 DONE WITH: 9.mp4


In [11]:
from collections import Counter

# Flatten the list of lists
flattened_list_accross_frames = [item for sublist in label_accross_frames_list for item in sublist]
flattened_list_with_delays = [item for sublist in label_with_delays_list for item in sublist]

# Count unique terms
counter_accross_frames = Counter(flattened_list_accross_frames)
counter_with_delays = Counter(flattened_list_with_delays)

# Print the counts
print("Counts for label_accross_frames_list:")
print(counter_accross_frames)

print("Counts for label_with_delays_list:")
print(counter_with_delays)

Counts for label_accross_frames_list:
Counter({'Extended Side Angle': 789, 'Corpse': 420, 'Cat': 352, 'Triangle': 310, 'Extended Hand to Toe': 227, 'Crow': 219, 'Sphinx': 111, 'Half-Moon': 49, 'Bridge': 23, 'Chair': 18, 'Low Lunge': 16, 'Warrior I': 12, 'Squat': 11, 'Crescent Lunge': 10, 'Plow': 10, 'Warrior III': 9, 'Forearm Stand': 8, 'Reverse Warrior': 6, 'Pigeon': 5, "Child's Pose": 5, 'Seated Forward Bend': 5, 'Warrior II': 4, 'Shoulder Stand': 3, 'Standing Forward Bend': 3, 'Butterfly': 1, 'Half-Boat': 1, 'Camel': 1, 'King Pigeon': 1})
Counts for label_with_delays_list:
Counter({'Extended Side Angle': 618, 'Corpse': 350, 'Cat': 333, 'Triangle': 271, 'Crow': 211, 'Extended Hand to Toe': 170, 'Sphinx': 77, 'Bridge': 20, 'Chair': 15, 'Half-Moon': 13, 'Warrior I': 8, 'Low Lunge': 8, 'Crescent Lunge': 7, 'Squat': 7, 'Forearm Stand': 5, 'Warrior III': 5, 'Plow': 4})
